In [1]:
import os
import numpy as np 
import pandas as pd 
import tensorflow as tf
import keras
import matplotlib.pyplot as plt

/home/gskim/anaconda3/envs/tfkeras/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [67]:
# Data info 
ICRArootDir = '/media/gskim/IRAP-ADV1/Data/ICRA2019/'

Dataset = 'NCLT'
TrainOrTest = '/Test/'
SequenceDate = '2013-04-05'

SCImiddlePath = '/4. SCI_jet0to15/'

GridCellSize = '10'

DataPath = ICRArootDir + Dataset + TrainOrTest + SequenceDate + SCImiddlePath + GridCellSize + '/'
print(DataPath)

/media/gskim/IRAP-ADV1/Data/ICRA2019/NCLT/Test/2013-04-05/4. SCI_jet0to15/10/


In [110]:
DataPath = ICRArootDir + Dataset + TrainOrTest + SequenceDate + SCImiddlePath + GridCellSize + '/'
print(DataPath)

/media/gskim/IRAP-ADV1/Data/ICRA2019/NCLT/Test/2012-09-28/4. SCI_jet0to15/5/


In [4]:
def getTestDataNCLT(DataPath, SequenceDate, lbl_enc_train):    

    # load training label encoding information 
    from sklearn.preprocessing import LabelEncoder
    ClassesTrainingSequenceHave = lbl_enc_train.classes_
    nClassesTrainingSequenceHave = len(ClassesTrainingSequenceHave)

    # info
    WholeData = os.listdir(DataPath)
    nWholeData = len(WholeData)
    print(str(nWholeData) + ' data exist in ' + SequenceDate)
    
    # read 
    X_seen = []
    y_seen = []
    X_unseen = []
    y_unseen = []
    
    for ii in range(nWholeData):
        dataName = WholeData[ii]
        dataPath = DataPath + dataName
        
        dataTrajNodeOrder = int(dataName[0:5])

        SCI = plt.imread(dataPath)
        dataPlaceIndex = int(dataName[6:11])
        
        # if label is in the train, then save into the seen (seen is only subset to be tested)
        if dataPlaceIndex in ClassesTrainingSequenceHave:
            X_seen.append(SCI)
            y_seen.append(dataPlaceIndex)
        else:
            X_unseen.append(SCI)
            y_unseen.append(dataPlaceIndex)
        
        # progress message 
        if ii%1000==0:
            print(str(format((ii/nWholeData)*100, '.1f')), '% loaded.')
        
    dataShape = SCI.shape
    
    # X
    nSeenData = len(X_seen)
    X_nd = np.zeros(shape=(nSeenData, dataShape[0], dataShape[1], dataShape[2]))
    for jj in range(nSeenData):
        X_nd[jj, :, :] = X_seen[jj]
    X_nd = X_nd.astype('float32')
    
    # y (one-hot encoded)   
    y_seen = lbl_enc_train.transform(y_seen)
    y_nd = keras.utils.np_utils.to_categorical(y_seen, num_classes=nClassesTrainingSequenceHave)

    # log message 
    print('Data size: %s' % nWholeData)
    print('- Seen data: %s' % len(X_seen))
    print('- Uneen data: %s' % len(X_unseen))
    print(' ')
    print('Data shape:', X_nd.shape)
    print('Label shape:', y_nd.shape)
    
    return X_nd, y_nd, X_unseen, y_unseen
    

In [63]:
# load training label encoding information for discriminate seen/unseen of test 
import pickle

TrainingDate = '2012-01-15'
TrainingDataPath = 'data_pickle/Train_' + TrainingDate + '_SCI_color.pkl'

with open(TrainingDataPath, 'rb') as f:  # Python 3: open(..., 'rb')
    X_train, y_train, lbl_enc_train = pickle.load(f)

In [73]:
# Load test data using training label encoder information  
[X_seen, y_seen, X_unseen, y_unseen] = getTestDataNCLT(DataPath, SequenceDate, lbl_enc_train)

5877 data exist in 2012-03-17
0.0 % loaded.
17.0 % loaded.
34.0 % loaded.
51.0 % loaded.
68.1 % loaded.
85.1 % loaded.
Data size: 5877
- Seen data: 4916
- Uneen data: 961
 
Data shape: (4916, 40, 120, 3)
Label shape: (4916, 1204)


In [74]:
import pickle

# Saving loaded array-form data for later fast load
fileName = 'data_pickle/Test_' + SequenceDate + '_SCI_color.pkl'
with open(fileName, 'wb') as f: 
    pickle.dump([X_seen, y_seen, X_unseen, y_unseen], f)

In [68]:
import pickle

# fast Load
fileName = 'data_pickle/Test_' + SequenceDate + '_SCI_color.pkl'
with open(fileName, 'rb') as f: 
    X_seen, y_seen, X_unseen, y_unseen = pickle.load(f)

In [69]:
SequenceDate

'2013-04-05'

In [70]:
dataShape = X_seen[0].shape
labelShape = y_seen[0].shape

print(dataShape)
print(labelShape)

(40, 120, 3)
(579,)


In [71]:
print(y_seen.shape)
print(X_seen.shape)
# print(y_unseen.shape)
print(np.array(X_unseen).shape)


(3341, 579)
(3341, 40, 120, 3)
(1162, 40, 120, 3)


In [1]:
from keras.models import load_model
modelName = 'pre_trained_model/base0.h5'
testModel = load_model(modelName)

testModel.summary()

/home/gskim/anaconda3/envs/tfkeras/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40, 120, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 120, 64)       4864      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 60, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 20, 60, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 60, 128)       204928    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 30, 128)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 10, 30, 128)       512       
__________

In [13]:
# Load Trained net 
from keras.models import load_model
modelName = 'model/base0.h5'
testModel = load_model(modelName)

# Predict 
scores_TEST = testModel.evaluate(X_seen, y_seen, verbose=1, batch_size=1000)
print(SequenceDate)
print('- Test score:', scores_TEST[0])
print('- Test accuracy:', scores_TEST[1]*100)
print(' ')

5170/5170 [==============================] - 1s 182us/step
2012-02-04
- Test score: 1.008079155962518
- Test accuracy: 82.76595741913904
 


In [21]:
# save prediction (for later top N analysis )
y_seen_predicted = testModel.predict(X_seen)

# save 
filename_y_seen_predicted = 'results_predictionvectors/base0/' + SequenceDate + '_seen_predicted'
np.save(filename_y_seen_predicted, y_seen_predicted)

# save prediction (for later top N analysis )
X_unseen = np.array(X_unseen)
y_unseen_predicted = testModel.predict(X_unseen)

# save 
filename_y_unseen_predicted = 'results_predictionvectors/base0/' + SequenceDate + '_unseen_predicted'
np.save(filename_y_unseen_predicted, y_unseen_predicted)

In [72]:
### save GT also 

# seen 
filename_y_seen_GT = 'results_predictionvectors/base0/' + SequenceDate + '_seen_GT'
np.save(filename_y_seen_GT, y_seen)

# unseen
filename_y_unseen_GT = 'results_predictionvectors/base0/' + SequenceDate + '_unseen_GT'
np.save(filename_y_unseen_GT, y_unseen)



In [30]:
# save success check 
filename_y_seen_gt = filename_y_seen_GT + '.npy'
y_seen_GT = np.load(filename_y_seen_gt)
print(y_seen_GT.shape)

filename_y_unseen_gt = filename_y_unseen_GT + '.npy'
y_unseen_GT = np.load(filename_y_unseen_gt)
print(y_unseen_GT.shape)

(5533, 579)
(773,)


In [97]:
SequenceDate

'2012-08-20'